- # Silver layer
    - ## Clean, deduplicate, typed, validated

In [ ]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession

builder = SparkSession.builder \
    .appName("LocalDeltaLake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
from pyspark.sql.functions import col, to_date

bronze_input = "../data/bronze/openpowerlifting_bronze.parquet"
silver_output = "../data/silver/openpowerlifting_silver.parquet"

def run_etl(input_path: str, output_path: str):
    df = spark.read.parquet(input_path)

    required_cols = ["Name", "Sex", "Event", "Equipment", "Age", "date"]

    df_clean = (
        df.na.drop(subset=required_cols)
          .withColumn("date", to_date(col("Date"), "yyyy-MM-dd"))  # Replace "Date" with actual column name
          .fillna({"Tested": "Untested"})  # Replaces nulls in "Tested" column
    )

    df_clean.write.format("parquet").mode("overwrite").save(output_path)

    return df_clean

df_silver = run_etl(bronze_input, silver_output)


# EDA

In [ ]:
df_silver.printSchema()

In [ ]:
print(df_silver.count())

In [ ]:
from pyspark.sql.functions import col, sum

df_silver.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in df_silver.columns
]).show()

# Univariate analysis

In [ ]:

pdf_sex_counts = df_silver.filter(df_silver["Sex"] != "Mx").groupBy("Sex").count().orderBy("count", ascending=False).toPandas()

import matplotlib.pyplot as plt

plt.figure(figsize=(6, 6))
plt.pie(
    pdf_sex_counts["count"],
    labels=pdf_sex_counts["Sex"],
    autopct="%1.1f%%",
    startangle=90,
    counterclock=False
)
plt.title("Sex Distribution in Dataset")
plt.tight_layout()

%matplotlib inline

# Save the plot as a PNG file
plt.savefig("../images/sex_split.png")
plt.show()


In [ ]:
#df_silver.select("Age").describe().show()
#df_silver.select("Age").groupBy("Age").count().orderBy("Age").show()

In [ ]:
ageclass_counts_df = df_silver.filter(df_silver["AgeClass"] != "NULL").groupBy("AgeClass").count()
pdf_ageclass = ageclass_counts_df.toPandas()

# Optional: remove nulls and sort logically
pdf_ageclass = pdf_ageclass[pdf_ageclass["AgeClass"].notnull()]
pdf_ageclass = pdf_ageclass.sort_values(by="AgeClass")

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.bar(pdf_ageclass["AgeClass"], pdf_ageclass["count"], color='skyblue')
plt.xticks(rotation=45)
plt.xlabel("Age Class")
plt.ylabel("Count")
plt.title("Distribution of Lifters by Age Class")
plt.tight_layout()

plt.savefig("../images/lifters_age.png")

plt.show()

# Bivariate analysis

# Lifter analysis
- Correlation between bodyweight and total (same as Weight class candlesticks per weight class men vs women)
- Raw vs equippend, tested vs untested percentile comparison
- Squat bench, Squat deadlift, Bench deadlift ratios + averages and grouping per weight class men vs women


In [ ]:
age_sex_counts_df = df_silver.groupBy("AgeClass", "Sex").count()

pdf_age_sex = age_sex_counts_df.toPandas()

# Optional: remove nulls and irrelevant entries like 'Mx'
pdf_age_sex = pdf_age_sex[pdf_age_sex["AgeClass"].notnull() & pdf_age_sex["Sex"].isin(["M", "F"])]

# Pivot table: rows = AgeClass, columns = Sex, values = count
pivot_df = pdf_age_sex.pivot(index="AgeClass", columns="Sex", values="count").fillna(0)

# Sort AgeClass numerically if it is in the form '20-24', '25-29', etc.
def age_sort_key(x):
    return int(x.split('-')[0]) if '-' in x else 0

# Step 1: Create the sort key function
def age_sort_key(x):
    return int(x.split('-')[0]) if '-' in x else 0

# Step 2: Reindex using the mapped sort key
pivot_df = pivot_df.loc[
    pivot_df.index.to_series()
    .map(age_sort_key)
    .sort_values()
    .index
]

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Plot stacked bars
plt.bar(pivot_df.index, pivot_df["M"], label="Male", color="steelblue")
plt.bar(pivot_df.index, pivot_df["F"], bottom=pivot_df["M"], label="Female", color="salmon")

plt.xticks(rotation=45)
plt.xlabel("Age Class")
plt.ylabel("Count")
plt.title("Lifter Count by Age Class and Sex")
plt.legend()
plt.tight_layout()

plt.savefig("../images/lifters_age_and_sex.png")

plt.show()

In [ ]:
df_silver.groupBy("BirthYearClass").count().orderBy("count", ascending=False).show()

In [ ]:
# Group by Event and get counts
event_counts = df_silver.groupBy("Event").count()

# Convert to Pandas
event_counts_pd = event_counts.toPandas()

# Drop nulls if needed
event_counts_pd = event_counts_pd.dropna()

# Plot pie chart
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
plt.pie(
    event_counts_pd["count"],
    labels=event_counts_pd["Event"],
    autopct="%1.1f%%",
    startangle=90
)
plt.title("Distribution of Events")
plt.axis("equal")  # Equal aspect ratio ensures the pie is circular.
plt.savefig("../images/event_type.png")
plt.show()


In [ ]:
equipment_count = df_silver.groupBy("Equipment").count()

# Convert to Pandas
equipment_count_pd = equipment_count.toPandas()

# Drop nulls if needed
equipment_count_pd = equipment_count_pd.dropna()

# Plot pie chart
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
plt.pie(
    equipment_count_pd["count"],
    labels=equipment_count_pd["Equipment"],
    autopct="%1.1f%%",
    startangle=90
)
plt.title("Distribution of Equipment")
plt.axis("equal")  # Equal aspect ratio ensures the pie is circular.
plt.savefig("../images/equipment_count.png")
plt.show()


In [ ]:
tested_df_silver= df_silver.groupBy("Tested").count()

# Convert to Pandas
tested_df_silver_pd = tested_df_silver.toPandas()

# Drop nulls if needed
tested_df_silver_pd = tested_df_silver_pd.dropna()

# Plot pie chart
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
plt.pie(
    tested_df_silver_pd["count"],
    labels=tested_df_silver_pd["Tested"],
    autopct="%1.1f%%",
    startangle=90
)
plt.title("Percent Tested vs Untested")
plt.axis("equal")  # Equal aspect ratio ensures the pie is circular.
plt.savefig("../images/tested_percent.png")
plt.show()


In [ ]:
df_silver.groupBy("Federation").count().orderBy("count", ascending=False).show(20)
# Get the top 20 ParentFederations (excluding nulls), convert to Pandas
pf_counts = (
    df_silver
    .filter(df_silver["ParentFederation"].isNotNull())  # filter out nulls
    .groupBy("ParentFederation")
    .count()
    .orderBy("count", ascending=False)
    .limit(20)  # top 20 only
)

pf_counts_pd = pf_counts.toPandas()

# Plot horizontal bar chart
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(pf_counts_pd["ParentFederation"], pf_counts_pd["count"], color="skyblue")
plt.xlabel("Count")
plt.ylabel("Parent Federation")
plt.title("Top 20 Parent Federations")
plt.gca().invert_yaxis()
plt.tight_layout()

plt.savefig("../images/parent_fed.png")

plt.show()


In [ ]:
df_silver.select("Best3SquatKg", "Best3BenchKg", "Best3DeadliftKg", "TotalKg","Dots", "Wilks").describe().show()

In [ ]:
pdf = df_silver.select("Best3SquatKg", "Best3BenchKg", "Best3DeadliftKg", "TotalKg","Dots", "Wilks").toPandas()
pdf.corr()

In [ ]:
df_silver.groupBy("Country").count().orderBy("count", ascending=False).show()

country_counts = (
    df_silver.filter(df_silver["MeetCountry"].isNotNull())
    .groupBy("MeetCountry")
    .count()
    .orderBy("count", ascending=False)
    .limit(20)
)
country_counts_pd = country_counts.toPandas()

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(country_counts_pd["MeetCountry"], country_counts_pd["count"], color="mediumseagreen")
plt.xlabel("Meet Count")
plt.ylabel("Country")
plt.title("Top 20 Countries by Meet Count")
plt.gca().invert_yaxis()
plt.tight_layout()

plt.savefig("../images/Meet_country.png")

plt.show()



In [ ]:
df_silver.groupBy("State").count().orderBy("count", ascending=False).show(20)
df_silver.groupBy("MeetState").count().orderBy("count", ascending=False).show(20)


In [ ]:
lifter_analysis_df = df_silver.select("Name","Sex","Equipment","Event", "AgeClass","Division","WeightClassKg","Best3SquatKg","Best3BenchKg","Best3DeadliftKg","TotalKg","Dots","Tested", "date")
#lifter_analysis_df.show()

In [ ]:
bw_total_df = df_silver.select("Sex", "TotalKg", "Event").filter(df_silver["Event"] == "SBD").toPandas()

In [ ]:
import matplotlib.pyplot as plt

# Filter SBD events and convert to pand
# Split data by gender (using pandas DataFrame, not PySpark)
male_totalkg = bw_total_df[bw_total_df["Sex"] == "M"]["TotalKg"].dropna()
female_totalkg = bw_total_df[bw_total_df["Sex"] == "F"]["TotalKg"].dropna()


# Define common bins
bins = 50

# Plot
plt.hist(male_totalkg, bins=bins, color='blue', alpha=0.5, label='Male')
plt.hist(female_totalkg, bins=bins, color='pink', alpha=0.8, label='Female')

plt.title('Distribution of TotalKg (SBD only)')
plt.xlabel('TotalKg')
plt.ylabel('Frequency')
plt.legend()
plt.tight_layout()

plt.savefig("../images/male_female_distributions.png")

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Split data
male_totalkg = bw_total_df[bw_total_df["Sex"] == "M"]["TotalKg"].dropna()
female_totalkg = bw_total_df[bw_total_df["Sex"] == "F"]["TotalKg"].dropna()

# Define bins
bins = 50

# Define percentiles to show
percentile_points = np.arange(0, 101, 10)

# Calculate percentiles and stats
def get_stats(data):
    percentiles = np.percentile(data, percentile_points)
    desc = data.describe()
    return percentiles, desc

male_percentiles, male_desc = get_stats(male_totalkg)
female_percentiles, female_desc = get_stats(female_totalkg)

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(18, 6), constrained_layout=True)

# --- Male plot ---
axes[0].hist(male_totalkg, bins=bins, color='blue', alpha=0.7)
axes[0].set_title("Male TotalKg Distribution")
axes[0].set_xlabel("TotalKg")
axes[0].set_ylabel("Frequency")

# Add percentile lines and labels
for p, value in zip(percentile_points, male_percentiles):
    axes[0].axvline(value, color='black', linestyle='--', linewidth=0.8)
    axes[0].text(value, axes[0].get_ylim()[1] * 0.95,
                 f"{p}th\n{int(round(value))}",
                 rotation=90, verticalalignment='top',
                 horizontalalignment='center', fontsize=8, color='black')

# Add male stats
stats_text = "\n".join([
    f"Count: {int(male_desc['count'])}",
    f"Mean: {male_desc['mean']:.0f}",
    f"Std: {male_desc['std']:.0f}",
    f"Min: {male_desc['min']:.0f}",
    f"25%: {male_desc['25%']:.0f}",
    f"50%: {male_desc['50%']:.0f}",
    f"75%: {male_desc['75%']:.0f}",
    f"Max: {male_desc['max']:.0f}",
])
axes[0].text(1.02, 0.5, stats_text,
             transform=axes[0].transAxes,
             fontsize=10, va='center')

# --- Female plot ---
axes[1].hist(female_totalkg, bins=bins, color='pink', alpha=0.7)
axes[1].set_title("Female TotalKg Distribution")
axes[1].set_xlabel("TotalKg")
axes[1].set_ylabel("Frequency")

# Add percentile lines and labels
for p, value in zip(percentile_points, female_percentiles):
    axes[1].axvline(value, color='black', linestyle='--', linewidth=0.8)
    axes[1].text(value, axes[1].get_ylim()[1] * 0.95,
                 f"{p}th\n{int(round(value))}",
                 rotation=90, verticalalignment='top',
                 horizontalalignment='center', fontsize=8, color='black')

# Add female stats
stats_text = "\n".join([
    f"Count: {int(female_desc['count'])}",
    f"Mean: {female_desc['mean']:.0f}",
    f"Std: {female_desc['std']:.0f}",
    f"Min: {female_desc['min']:.0f}",
    f"25%: {female_desc['25%']:.0f}",
    f"50%: {female_desc['50%']:.0f}",
    f"75%: {female_desc['75%']:.0f}",
    f"Max: {female_desc['max']:.0f}",
])
axes[1].text(1.02, 0.5, stats_text,
             transform=axes[1].transAxes,
             fontsize=10, va='center')

plt.savefig("../images/male_female_distributions_percentiles.png")

plt.show()


# EDA per bw

In [ ]:
#bw_total_df.show()
#bw_total_df.count()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Load and clean data
df = lifter_analysis_df.select(
    "Name", "Sex", "Equipment", "WeightClassKg", "TotalKg"
).na.drop().toPandas()
df = df[df["Equipment"] == "Raw"]
df["WeightClassKg"] = pd.to_numeric(df["WeightClassKg"], errors="coerce")
df = df.dropna(subset=["WeightClassKg", "TotalKg"])

# Define custom binning function
def assign_weight_bin(row):
    wc = row["WeightClassKg"]
    if row["Sex"] == "M":
        if wc >= 140:
            return "140kg+"
        else:
            return f"{(wc // 10) * 10:.0f}-{(wc // 10) * 10 + 9:.0f}kg"
    elif row["Sex"] == "F":
        if wc >= 100:
            return "100kg+"
        else:
            return f"{(wc // 10) * 10:.0f}-{(wc // 10) * 10 + 9:.0f}kg"
    return None

df["WeightBin"] = df.apply(assign_weight_bin, axis=1)

# Define plotting function
def plot_sex_percentiles(sex_label):
    sex_df = df[df["Sex"] == sex_label]
    bin_labels = sorted(sex_df["WeightBin"].unique(), key=lambda x: float(x.rstrip("+kg").split("-")[0]))
    cmap = cm.get_cmap("rainbow")
    n_bins = len(bin_labels)
    percentiles = np.arange(0, 101, 10)

    plt.figure(figsize=(12, 7))
    for i, b in enumerate(bin_labels):
        bin_df = sex_df[sex_df["WeightBin"] == b]
        if len(bin_df) < 10:
            continue
        pct_values = np.percentile(bin_df["TotalKg"], percentiles)
        color = cmap(i / max(n_bins - 1, 1))
        plt.plot(percentiles, pct_values, label=b, color=color)
        plt.scatter(percentiles, pct_values, color=color, s=30)
        for x, y in zip(percentiles, pct_values):
            plt.text(x, y, f"{int(y)}", fontsize=8, ha="center", va="bottom", color=color)

    title = f"TotalKg Percentiles by Weight Bin ({'Men' if sex_label == 'M' else 'Women'}, Raw)"
    plt.title(title)
    plt.xlabel("Percentile")
    plt.ylabel("TotalKg")
    plt.grid(True)
    plt.legend(title="Weight Bin", fontsize=8, loc="upper left", bbox_to_anchor=(1, 1))
    plt.tight_layout()

    plt.savefig("../images/TotalKg_by_percentiles_weight_bin_women.png")
    
    plt.show()

# Generate charts
plot_sex_percentiles("M")  # Men
plot_sex_percentiles("F")  # Women


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assume bw_total_df is already loaded
bw_total_df = lifter_analysis_df.select("Name", "Sex", "Equipment", "WeightClassKg", "TotalKg").na.drop().toPandas()

# Keep only Raw equipment
bw_total_df = bw_total_df[bw_total_df["Equipment"] == "Raw"]

# Convert WeightClassKg to numeric (in case of string like "83+" or "120+")
bw_total_df["WeightClassKg"] = pd.to_numeric(bw_total_df["WeightClassKg"], errors='coerce')

# Drop any rows with NaNs after conversion
bw_total_df = bw_total_df.dropna(subset=["WeightClassKg", "TotalKg"])

# IPF weight class binning
def ipf_bin(row):
    sex = row["Sex"]
    wc = row["WeightClassKg"]
    if sex == "M":
        if wc <= 59: return "59"
        elif wc <= 66: return "66"
        elif wc <= 74: return "74"
        elif wc <= 83: return "83"
        elif wc <= 93: return "93"
        elif wc <= 105: return "105"
        elif wc <= 120: return "120"
        else: return "120+"
    elif sex == "F":
        if wc <= 47: return "47"
        elif wc <= 52: return "52"
        elif wc <= 57: return "57"
        elif wc <= 63: return "63"
        elif wc <= 69: return "69"
        elif wc <= 76: return "76"
        elif wc <= 84: return "84"
        else: return "84+"
    else:
        return None

# Apply binning
bw_total_df["IPF_WeightClass"] = bw_total_df.apply(ipf_bin, axis=1)

# Percentiles to compute
percentiles = np.arange(0, 110, 10)

def plot_percentile_lines(ax, data, color):
    perc_values = np.percentile(data, percentiles)
    for p, val in zip(percentiles, perc_values):
        ax.axhline(val, linestyle="--", color=color, alpha=0.4)
        ax.text(x=0.5, y=val, s=f"{p}th: {int(round(val))}", color=color,
                fontsize=8, alpha=0.8, va='bottom', ha='left', transform=ax.get_yaxis_transform())

# Plot for each sex
for sex in ["M", "F"]:
    fig, ax = plt.subplots(figsize=(10, 6))
    subset = bw_total_df[bw_total_df["Sex"] == sex]
    classes = sorted(subset["IPF_WeightClass"].unique(), key=lambda x: float(x.replace('+', '')))

    # rainbow color palette
    palette = sns.color_palette("hsv", len(classes))

    for i, wc in enumerate(classes):
        class_data = subset[subset["IPF_WeightClass"] == wc]["TotalKg"]
        if len(class_data) < 2:
            continue
        percs = np.percentile(class_data, percentiles)
        ax.plot(percentiles, percs, label=wc, color=palette[i])
        for p, val in zip(percentiles, percs):
            ax.scatter(p, val, color=palette[i], s=20)
            ax.text(p, val, f"{int(val)}", fontsize=7, alpha=0.8, color=palette[i], ha='center', va='bottom')

    ax.set_title(f"Percentile Trends by IPF Weight Class - {'Men' if sex == 'M' else 'Women'} (Raw Only)")
    ax.set_xlabel("Percentile")
    ax.set_ylabel("TotalKg")
    ax.legend(title="Weight Class", bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True)

    # Draw horizontal percentile lines across all classes
    plot_percentile_lines(ax, subset["TotalKg"], "black")

    plt.tight_layout()
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Filter to Raw Men only
male_df = bw_total_df[(bw_total_df["Sex"] == "M") & (bw_total_df["Equipment"] == "Raw")]

# Apply IPF binning again (if not already applied)
def ipf_bin(row):
    wc = row["WeightClassKg"]
    if wc <= 59: return "59"
    elif wc <= 66: return "66"
    elif wc <= 74: return "74"
    elif wc <= 83: return "83"
    elif wc <= 93: return "93"
    elif wc <= 105: return "105"
    elif wc <= 120: return "120"
    else: return "120+"

male_df["IPF_WeightClass"] = male_df.apply(ipf_bin, axis=1)

# Get only 120+ kg class
target_class_df = male_df[male_df["IPF_WeightClass"] == "105"]

# Ryan_Cotter's total
Ryan_Cotter = 712

# Compute percentile
percentile = np.mean(target_class_df["TotalKg"] < Ryan_Cotter) * 100

# Plot histogram
plt.figure(figsize=(10, 6))
sns.histplot(target_class_df["TotalKg"], bins=30, color="skyblue", kde=True)

# Add vertical line for Ryan_Cotter
plt.axvline(Ryan_Cotter, color="red", linestyle="--", linewidth=2)
plt.text(Ryan_Cotter, plt.ylim()[1]*0.9,
         f"Ryan Cotter\n{int(Ryan_Cotter)}kg\n{percentile:.1f}th percentile",
         color="red", ha="left", va="top", fontsize=10, backgroundcolor="white")

# Labels
plt.title("TotalKg Distribution in IPF 105kg (Men, Raw)")
plt.xlabel("TotalKg")
plt.ylabel("Lifter Count")

plt.tight_layout()
plt.show()


# Bivariate analysis
- Somehow all lifters show up as tested

In [ ]:
testing_equipment_df = df_silver.select("Name", "Sex", "WeightClassKg", "TotalKg", "Tested", "Equipment").na.drop().toPandas()
#testing_equipment_df.show()
#testing_equipment_df.count()

In [ ]:
plt.scatter(df['WeightClassKg'], df['TotalKg'], alpha=0.5)
plt.title('WeightClassKg vs TotalKg')
plt.xlabel('WeightClassKg')
plt.ylabel('TotalKg')
plt.show()

In [ ]:
lift_ratios = lifter_analysis_df.select("Name", "Sex", "WeightClassKg", "Best3SquatKg", "Best3BenchKg", "Best3DeadliftKg").na.drop().toPandas()
#lift_ratios.show()
#lift_ratios.count()

In [ ]:
age_analysis_raw = lifter_analysis_df.select("Name","Sex","Equipment","AgeClass","WeightClassKg","TotalKg","Tested").na.drop().toPandas()
#age_analysis_raw.show()
#age_analysis_raw.count()


# Time series df
- progression of records per weightclass over time per equipment and testing status

In [ ]:
records_progression = lifter_analysis_df.select("Name","Sex","Equipment","Division","WeightClassKg","TotalKg","Tested", "date").toPandas()
#records_progression.show()
#records_progression.count()